***1. Importing Libraries***

In [178]:
import pandas as pd
import numpy as np
import os

***2. Importing Data***

In [180]:
#create path
path=r'C:\Users\lisac\10-10-2024 Instacart Basket Analysis'

In [181]:
#import wrangled data as orders_wrangled csv
df_ords=pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'))

***3. Perform Data Consistency Checks***

In [183]:
#Task 2: Run .describe() function on df_ords dataframe and share in markdown cell whether anyting about the data looks off or should be investigated further.
df_ords.describe()

,Unnamed: 0,order_id,customer_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [184]:
#Dropping "Unnamed: 0", "eval_set" and "order_day_of_week" as these columns are not needed
df_ords=df_ords.drop(['Unnamed: 0', 'eval_set', 'order_day_of_week'], axis=1)

In [185]:
#Running .describe() function again to see if "Unnamed:0", eval_set', "order_day_of-Week" were dropped:
df_ords.describe()

,order_id,customer_id,order_number,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,2.300000e+01,3.000000e+01


Observation:  Three columns were dropped, but they should not have come up in the first place.  I retrieved the "orders_wrangled.csv" for the df_ords dataframe. 

Review of each column:
order_id: This is a count of each order.  The first order is 1 (also the minimum value) and the last order is 3421083 (also the maximun value).  The maximum value also matches the count, 3421083.

customer_id: This column is a count of customers, ranging from 1 to 206209.  Could be accurate for a count of 206209 customers.  Will check for duplicates.

order_number:  This column is a count of number of orders placed by customers; ranging from 1-100.  The average (mean) number of orders placed by a customer is 17.  Could look into high number of orders (90-100) to see if they are perhaps automated ordering.

order_hour_of_day:  This column is the time (hour of day) when orders were placed by customers.  Range from 0 hour (midnight) to 23:00 (or 11 pm).  The mean is 13:00 (1pm) which could track with most customers placing orders in the afternoon. Could investigate the distribution of order times to see if any hours are overrepresented or if there are unusual patterns in ordering times.

days_since_prior_order:  This column is a tally of the number of days from one order to the next.
The count is 3,214,874, which is is less than the total number of rows, 3,421,083.  This could mean there are missing values in this column. 

***3.1 Identify Mixed-Type Data***

In [188]:
#Task 3: Check for mixed-type data in df_ords dataframe by Looping over all columns to identify mixed-type data
mixed_type_data = []

for col in df_ords.columns:
    if (df_ords[[col]].apply(lambda x: x.map(type)) != df_ords[[col]].iloc[0].apply(type)).any(axis=None):
        mixed_type_data.append(col)

if mixed_type_data:
    print(f"Columns with mixed-type data: {mixed_type_data}")
else:
    print("No mixed-type data detected.")

No mixed-type data detected.


#Task 4:  If any mixed-type data found, fix it.  
No mixed-type data found

***3.2 Identify Missing Values***

In [191]:
#Task 5: Check for missing values in df_ords dataframe and report findings
df_ords.isnull().sum()

order_id                       0
customer_id                    0
order_number                   0
order_hour_of_day              0
days_since_prior_order    206209
order_day_of_week_name         0
dtype: int64

Observation: there are 206209 missing values in the days_since_prior_order column.  This tracks because that is exactly the difference between the total count and the total number of days since the last order.  

In [193]:
#Task 6:  Addressing missing values by first creating a subset to review the missing values
df_nan=df_ords[df_ords['days_since_prior_order'].isnull()==True]

In [194]:
df_nan

,order_id,customer_id,order_number,order_hour_of_day,days_since_prior_order,order_day_of_week_name
0,2539329,1,1,8,NaN,Tuesday
11,2168274,2,1,11,NaN,Tuesday
26,1374495,3,1,14,NaN,Monday
39,3343014,4,1,11,NaN,Saturday
45,2717275,5,1,12,NaN,Wednesday
...,...,...,...,...,...,...
3420930,969311,206205,1,12,NaN,Thursday
3420934,3189322,206206,1,18,NaN,Wednesday
3421002,2166133,206207,1,19,NaN,Saturday
3421019,2227043,206208,1,15,NaN,Monday


Observation (1) My thought process:  I'm not sure which method to use to address these missing values.  The example in the reading was missing values that were a string ("product_name").  This could have been flagged, I guess, but the reading identified filtering out the missing rows as the method of choice.  Additionally, since they were string values, they could not be imputed.  I could understand filtering out the values as it was only 16 rows and it would not have a significant impact on the overall analysis.  

In this instance, the df_ords dataframe has 206209 missing values in the days_since_prior_order column.  I need to do some deeper investigating.

Observation (2) Investigative Action steps:  I took a deep dive into ChatGPT to help me figure out how to "think" about these missing numbers. I'm looking for these steps to help to deduce which method I should use (Flagging, Imputing or Filtering out) to address these missing values.   

Missing Values - What could they represent?:

Since days_since_prior_order reflects the time elapsed between orders for each customer. Missing values may suggest that these are first-time orders, meaning there is no prior order to calculate a time lapse.

Cross-Check Other Columns:

Customer ID: Check if these customers appear only once in dataset, which might indicate a first order.

Order Number: Check the value of order_number for all the missing rows. If it is 1, then it's likely this is a customer’s first order, confirming the hypothesis.

Statistical Analysis:

The 206209 rows are roughly 6% of the total number 3421083.  This percentage may not be large enough to have an impact on the overall analysis, thereby making it possible to just filter out these values.  But before that, how about counting and grouping?

Count the unique coustomer_id values where the days_since_prior_order is NaN to see how many distinct customers are involved.

Group the data by customer_id to see the order history and look for any patterns like if NaN is consistently associated with first time orders.

In [197]:
#Cross-Check customer_id:  Count number of unique customers with missing 'days_since_prior_order' in the 'cusotmer_id' column
unique_customers_with_nan = df_ords[df_ords['days_since_prior_order'].isnull()]['customer_id'].nunique()
print(f"Number of unique customers with missing values: {unique_customers_with_nan}")

Number of unique customers with missing values: 206209


Observation: There are 206209 unique customers with missing values.  Since these customers appear only once in the dataset, it's possible that these could be first time orders. Next, I'm going to look at the order_numbers to see if they are different or all "1".

In [199]:
#Sample some customers with missing 'days_since_prior_order' values
sample_missing_customers = df_ords[df_ords['days_since_prior_order'].isnull()].sample(5)['customer_id']
df_ords[df_ords['customer_id'].isin(sample_missing_customers)]

,order_id,customer_id,order_number,order_hour_of_day,days_since_prior_order,order_day_of_week_name
488047,1979389,29446,1,19,NaN,Tuesday
488048,713681,29446,2,16,13.0,Monday
488049,825355,29446,3,20,12.0,Saturday
488050,1222089,29446,4,16,18.0,Wednesday
488051,2992236,29446,5,16,30.0,Tuesday
...,...,...,...,...,...,...
3118875,389419,188036,46,8,10.0,Tuesday
3220695,2074108,194160,1,14,NaN,Sunday
3220696,335651,194160,2,13,18.0,Thursday
3220697,1000085,194160,3,9,7.0,Thursday


Observation: Customer_ids that have a "1" in the order_number column and "NaN" in the days_since_prior_order column indicate that these are first time orders. 

In [201]:
#Check by counting order_number distribution for rows with missing values
order_number_distribution = df_ords[df_ords['days_since_prior_order'].isnull()]['order_number'].value_counts()
print(order_number_distribution)

order_number
1    206209
Name: count, dtype: int64


Observation: This code is counting how many times an instance of "1" in the order_number column appears for the customers with "NaN" in the days_since_prior_order column.  It appears 206209 times, further supporting the hypothesis that these are first time orders.  

Additionally, this is a strong indicator that I could impute the value to "0" because "NaN" looks to be representing a first time order.  Next, I want to take a deeper dive into the customer_id.

In [203]:
# Step 1: Filter data where 'days_since_prior_order' is NaN
df_missing = df_ords[df_ords['days_since_prior_order'].isnull()]

# Step 2: Count occurrences of 'customer_id' for these rows
customer_counts = df_missing['customer_id'].value_counts()

# Step 3: Identify customers who appear only once
customers_appearing_once = customer_counts[customer_counts == 1].index

# Optional: View these customers
print(customers_appearing_once)

Index([     1, 137478, 137468, 137469, 137470, 137471, 137472, 137473, 137474,
       137475,
       ...
        68735,  68736,  68737,  68738,  68739,  68740,  68741,  68742,  68743,
       206209],
      dtype='int64', name='customer_id', length=206209)


Observation: This output shows a list (index) of customer_ids that appear only once among the rows with missing values in the days_since_prior_order column.  I noticed that the length is exactly 206209 rows.  This indicates that each customer with a missing value in the days_since_prior_order column appears only one time in the filtered subset, df_missing.  

Deduction:  It suggests that all the these customers made an order in the dataset.  Since orders PRIOR TO a FIRST TIME ORDER would be missing, it tracks that NaN would be the equivalent of a first time order.

How to address the NaN values:  Imputing a value of "0" would indicate a first time order.  This option allows retention of the data integrity.  It also allows for future analysis on the first time orders, should Instacart require it.

In [205]:
# Verify if these customers only appear once in the entire df_ords dataframe
total_customer_counts = df_ords['customer_id'].value_counts()
customers_appearing_once_total = total_customer_counts[total_customer_counts == 1].index

# Check intersection of customers appearing once in NaNsubset and entire dataframe
first_time_customers = set(customers_appearing_once).intersection(set(customers_appearing_once_total))

print(f"Number of first-time customers: {len(first_time_customers)}")

Number of first-time customers: 0


Observation:  I used the first code to verify if the customers who have missing days_since_prior_order values and appear only once in that filtered subset are also appearing only once in the entire DataFrame df_ords. Next, I looked at the commonality (intersection) of first_time_customers and customers_appearing_once to see if the first_time_customers contain any customers who are present only once in both conditions.  

The output for the number of first-time customers was "0".  This means that none of the customers with missing days_since_prior_order values and appearing only once in the filtered subset (df_missing) appear exactly once in the entire DataFrame df_ords.

In other words, all these customers have at least one other order recorded in df_ords.  This observation further supports that the "NaN" value in the days_since_prior_order column means that this was a first order for an existing customer, not a first order for a new customer.  This indicates a prior order history for these customers that are not present in the df_ords dataframe.

Recommended Action: Flagging and Imputing
Given that the missing days_since_prior_order values correspond to customers with multiple orders and that these NaN values likely represent their first recorded orders in this dataset, flagging these values as "is_first_order" and imputing "0" for the NaN value will give a clear picture of what the imputed "0" value is:  the first order for this existing customer in this dataset.

In [207]:
#Create a flag for missing 'days_since_prior_order'
df_ords['is_first_order'] = 0  # Initialize column with 0
df_ords.loc[df_ords['days_since_prior_order'].isnull(), 'is_first_order'] = 1

#Impute missing values in 'days_since_prior_order' with 0
df_ords['days_since_prior_order'] = df_ords['days_since_prior_order'].fillna(0)

In [208]:
#Checking to see if flag and imputation worked
#Check for the existence and values of the flag
print('is_first_order' in df_ords.columns)
print(df_ords['is_first_order'].value_counts())
print(df_ords[df_ords['is_first_order'] == 1].head())

#Check for successful imputation of 0 values
print(df_ords['days_since_prior_order'].isnull().sum())  # Should output 0
print(df_ords['days_since_prior_order'].value_counts().head())

#Cross-check consistency
check_consistency = df_ords[df_ords['is_first_order'] == -1]['days_since_prior_order'].eq(0).all()
print(f"Consistency between flag and imputed values: {check_consistency}")

True
is_first_order
0    3214874
1     206209
Name: count, dtype: int64
    order_id  customer_id  order_number  order_hour_of_day  \
0    2539329            1             1                  8   
11   2168274            2             1                 11   
26   1374495            3             1                 14   
39   3343014            4             1                 11   
45   2717275            5             1                 12   

    days_since_prior_order order_day_of_week_name  is_first_order  
0                      0.0                Tuesday               1  
11                     0.0                Tuesday               1  
26                     0.0                 Monday               1  
39                     0.0               Saturday               1  
45                     0.0              Wednesday               1  
0
days_since_prior_order
30.0    369323
7.0     320608
0.0     273964
6.0     240013
4.0     221696
Name: count, dtype: int64
Consistency between f

Observations:  
(1) Check for the existence of "is_first_order" column

The first line in the output "True" confirms that "is_first_order" column exists in the df_ords dataframe.

(2) Counts of the values in "is_first_order" column

There are 3214874 records where "is_first_order" is "0" meaning NOT a first order.  There are 206209 records where "1" is in "is_first_order" column meaning it IS a first order.

(3) Showing the first few rows where "is_first_order"is "1"

The other relevant columns are ""order_id", "customer_id", "days_since_prior_order" and "is_first_order".  Note that in each row, "days_since_prior_order" is "0", which confirms these rows represent first orders as expected.

(4) Check for null values in "days_since_prior_order"

Here, a "0" indicates that there are no longer missing values in "days_since_prior_order" column, confirming that all the missing values were successfully filled.

(5) Value counts in "days_since_prior_order"

This output shows the counts of the top values in "days_since_prior_order".  The value "30" appears 369323 times, the value "7" appears 320608 times and the value "0" appears 273964 times.  I noticed that this was an increase of 67755 over the original count of 206209.  This could mean that there were already 67755 zeros in the dataset prior to the imputation.

(6) Consistency check between flag and imputed value = True

This confirms that all the rows where "is_first_order" is "1" also have the "days_since_prior_order" equal to "0", indicating that the flagging and the imputation are consistent and correctly implemented.

In [210]:
#Checking to see if flag and imputation of "0" for the NaN values were implemented
df_ords.isna().sum()

order_id                  0
customer_id               0
order_number              0
order_hour_of_day         0
days_since_prior_order    0
order_day_of_week_name    0
is_first_order            0
dtype: int64

Observation: 
Using .isna() and .sum() functions to count the missing values per column.  

Since all the counts were "0", this means that there are NO MISSING values (NaN) in any of the columns listed. If further confirms that any missing values that were originally present in "days_since_prior_order" have been successfully filled in with "0".  It also confirms that the other columns in the df_ords dataframe have no missing data as well.

In [234]:
#Task 7: Checking for duplicates by creating "clean" copy of the df_ords dataframe
df_ords_clean = df_ords.copy()

In [236]:
#Checking for full duplicates
df_dups = df_ords_clean[df_ords_clean.duplicated()]

In [238]:
#Pulling up any full duplicates
df_dups

,order_id,customer_id,order_number,order_hour_of_day,days_since_prior_order,order_day_of_week_name,is_first_order


Observation:  There were no rows in the output, therefore, there are no full duplicates in df_ords_clean dataframe.

In [245]:
#Task 8:  No need to address full duplicates because there aren't any in the df_ords_clean dataframe.

In [247]:
#Create new subset dataframe that indicates the df_ords dataframe is clean with no duplicates
df_ords_clean_no_dups=df_ords_clean.drop_duplicates()

In [249]:
#Count the number of rows
df_ords_clean_no_dups.shape

(3421083, 7)

Observation:  Used the .shape function to confirm the number of rows and columns.  The count of the rows is 3421083, which matches the original count done at the beginning of this task. The number of columns increased by one when i added a flag column "is_first_order".

In [254]:
#Task 9: Export new datafrome to Prepared Data folder:
df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'))